<a href="https://colab.research.google.com/github/SananSuleymanov/ONNX_optimization/blob/main/optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 12.0 MB/s eta 0:00:00


In [ ]:
!pip install onnxruntime-tools

In [ ]:
!pip install tensorflow-model-optimization

In [5]:
!pip install onnx-tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.6 MB/s eta 0:00:00


In [3]:
!!pip install onnxoptimizer

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting onnxoptimizer',
 '  Downloading onnxoptimizer-0.3.8-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (647 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/647.0 KB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━\x1b \x1b327.7/647.0 KB\x1b \x1b9.5 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b647.0/647.0 KB\x1b \x1b11.6 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hCollecting onnx',
 '  Downloading onnx-1.13.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/13.5 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b4.3/13.5 MB\x1b \x1b130.0 MB/s\x1b eta \x1b0:00:01\x1b',

#Required Dependicies

In [6]:
from onnxruntime.quantization import quantize_dynamic, QuantType, quantize_static, CalibrationDataReader, preprocess
from onnxruntime.quantization import QuantType
from onnxoptimizer import optimize
import onnxruntime
from PIL import Image
import tensorflow as tf
import onnx_tf.backend as backend
import onnx 
import numpy as np
import time
import os

#Inference before optimization

In [7]:
model_path= '/content/drive/MyDrive/YoloV5_small/small_yolov5.onnx'

session= onnxruntime.InferenceSession(model_path)

input_name= session.get_inputs()[0].name
output_name= session.get_outputs()[0].name

In [8]:
input_shape= session.get_inputs()[0].shape
output_shape= session.get_outputs()[0].shape
print('Input shape: {}'.format(input_shape))
print('Output shape: {}'.format(output_shape))

Input shape: [1, 3, 1024, 1024]
Output shape: [1, 73728, 7]


In [9]:
image_path= '/content/drive/MyDrive/YoloV5_small/Calibration/Copy of Black__A__Pikl__0_3__L_Keyless__2__11-07-08__0.jpg'

inf_image= Image.open(image_path)
new_shape=(1024, 1024)

#preprocessing of single image for running inference
def preprocess(image):
  image= image.resize(new_shape)
  image= np.array(image, dtype=np.float32)
  image= image/255.0
  image= np.reshape(image, (image.shape[2], image.shape[0], image.shape[1]))
  image= np.expand_dims(image, axis=0)
  return image


In [10]:
input_data = preprocess(inf_image)

In [11]:
start_time= time.time()
output= session.run([output_name], {input_name: input_data} )
end_time= time.time()

inference_time= (end_time-start_time)
print('Inference speed before purning and quantization: {}'.format(inference_time))

Inference speed before purning and quantization: 1.024125337600708


#Preprocess Calibration Data

In [12]:
def data_reader(image_path):
  images=[]
  for filename in os.listdir(image_path):
    f= os.path.join(image_path, filename)
    image= Image.open(f)
    image= image.resize(new_shape)
    image= np.array(image, dtype=np.float32)
    image= image/255.0
    image= np.reshape(image, (image.shape[2], image.shape[0], image.shape[1]))
    image= np.expand_dims(image, axis=0)
    images.append(image)
  images= np.concatenate(
        np.expand_dims(images, axis=0), axis=0)
  return images

In [13]:
calib_image=data_reader('/content/drive/MyDrive/YoloV5_small/Calibration')

In [14]:
print('Calibration dataset size: {}'.format(calib_image.shape))

Calibration dataset size: (56, 1, 3, 1024, 1024)


#Save ONNX model as TF model

In [15]:
onnx_model = onnx.load(model_path)

# Convert ONNX model to TensorFlow format
tf_model = backend.prepare(onnx_model)
tf_model.export_graph("model_1/") 


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [16]:
converter= tf.lite.TFLiteConverter.from_saved_model('model_1/')
converter.optimizations= [tf.lite.Optimize.DEFAULT]
tflite_quant_model= converter.convert() 

In [17]:
with open('model_1/model.tflite', 'wb') as f:
  f.write(tflite_quant_model)

In [18]:
interpreter = tf.lite.Interpreter(model_path='/content/model_1/model.tflite')

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.allocate_tensors()

#preprocess image
image= Image.open('/content/drive/MyDrive/YoloV5_small/Calibration/Copy of Black__A__Pikl__0_3__L_Keyless__2__11-07-08__0.jpg')
image= image.resize(new_shape)
np_features= np.array(image)
np_features= np_features/255.0
np_features= np.reshape(np_features, (np_features.shape[2], np_features.shape[0], np_features.shape[1]))
input_type = input_details[0]['dtype']

if input_type == np.int8:
    input_scale, input_zero_point = input_details[0]['quantization']
    print("Input scale:", input_scale)
    print("Input zero point:", input_zero_point)
    print()
    np_features = (np_features / input_scale) + input_zero_point
    np_features = np.around(np_features)
    
# Convert features to NumPy array of expected type
np_features = np_features.astype(input_type)

# Add dimension to input sample (TFLite model expects (# samples, data))
np_features = np.expand_dims(np_features, axis=0)

interpreter.set_tensor(input_details[0]['index'], np_features)

# Run inference
start_time2= time.time()
interpreter.invoke()
end_time2= time.time()

inference_time2= (end_time2-start_time2)
print('Inference speed after quantization: {}'.format(inference_time2))


Inference speed after quantization: 1.6335320472717285


#Dynamic Quantization


Model is dynamic quantized and saved as 'model_quant.onnx' file

In [24]:
model_quant= 'model_quant.onnx'
quantized_model = quantize_dynamic(model_path, model_quant, weight_type=QuantType.QUInt8)


In [25]:
# Load the ONNX model
model = onnx.load('/content/model_quant.onnx')

# Define the magnitude threshold for pruning weights
threshold = 0.5

# Prune the model using onnxoptimizer
passes = ["extract_constant_to_initializer", "eliminate_unused_initializer"]
optimized_model = optimize(model, passes)

# Save the pruned model to a file
onnx.save(optimized_model, 'pruned_model.onnx')


#Inference after quantization and pruning

I couldn't run inference in the latest model because of the belowmentioned model

In [30]:
session= onnxruntime.InferenceSession('/content/pruned_model.onnx')

start_time3= time.time()
session= onnxruntime.InferenceSession('/content/pruned_model.onnx')
end_time3= time.time()

inference_time3= (end_time3-start_time3)
print('Inference speed after quantization and pruning: {}'.format(inference_time3))


Inference speed after quantization and pruning: 0.0789027214050293
